import sys
!{sys.executable} -m pip install xlsxwriter
!{sys.executable} -m pip install azure-storage-blob==12.3.1

In [ ]:
from PIL import Image, ImageDraw, ImageFont
import cv2, io, json, logging, os, sys, tempfile,uuid
import numpy as np
import xlsxwriter 
import matplotlib.pyplot as plt
%matplotlib inline
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateEntry, Region
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from matplotlib.patches import Rectangle
from datetime import datetime, timedelta
from azure.storage.blob import ResourceTypes, AccountSasPermissions, AccessPolicy, ContainerSasPermissions,generate_container_sas,BlobServiceClient, BlobClient, ContainerClient

In [ ]:
# endpoint for the personal training
endpoint = "https://<enter name here>.cognitiveservices.azure.com/"

# Replace with a valid key
prediction_key = "¯\_(ツ)_/¯"
projectID = "🙃"

# Replace with a valid iteration 
iteration = "Iteration9"

# Replace with a valid file path on your desktop with videos
path = "C:\\2019\Channel Billabong\\Sandy Billabong\\Transect 4\\Location 10"
# Storage connection string format "DefaultEndpointsProtocol=https;AccountName=audiostores2tg********"
connect_str = "DefaultEndpointsProtocol=htt*****"

# dictionary list for thresholding prediction values from Custom Vision
thresholding_values = {
    "Ambassis agrammus":50 ,
    "Ambassis macleayi":50 ,
    "Amniataba percoides":50 ,
    "Craterocephalus sturcusmuscarum":50,
    "Denariusa bandata":50,
    "Glossamia aprion":50,
    "Glossogobius spp.": 50,
    "Hephaestus fuliginosus": 50,
    "Lates calcarifer":50,
    "Leiopotherapon unicolor": 50,
    "Liza ordensis": 50,
    "Megalops cyprinoides": 50,
    "Melanotaenia nigrans":50 ,
    "Melanotaenia splendida inornata":50,
    "Mogurnda mogurnda": 50,
    "Nemetalosa erebi": 50,
    "Neoarius spp.":50,
    "Neosilurus spp.":50,
    "Oxyeleotris spp.": 50,
    "Other":50,
    "Scleropages jardinii":50,
    "Strongylura krefftii":50,
    "Sycomistes butleri":50,
    "Toxotes chatareus":50 
}


In [ ]:
def get_file_info(file_path):

    file_info = {}

    parts = file_path.split(os.sep)

    file_info['video_name'] = os.path.splitext(parts[-1])[0]
    file_info['location_name'] = parts[-2]
    file_info['transect_name'] = parts[-3]
    file_info['site_name'] = parts[-4]
    file_info['billabong_type'] = parts[-5]
    file_info['year'] = parts[-6]

    return file_info


In [ ]:
def draw_bounding_boxes(results, np_image, frame_count, fps, min_probability, debug=False):
    print(results.id)
    
    for prediction in results.predictions:
        if prediction.tag_name not in thresholding_values:
            print ("WARNING: The Species name is not in threshold dictionary, probability is set to default")
            probability = min_probability
        else:
            probability = thresholding_values[prediction.tag_name]

        if (prediction.probability * 100) > probability:
            x1 = int(prediction.bounding_box.left * np_image.shape[1])
            y1 = int(prediction.bounding_box.top * np_image.shape[0])
            x2 = x1 + int(prediction.bounding_box.width * np_image.shape[1])
            y2 = y1 + int(prediction.bounding_box.height * np_image.shape[0])
        
            cv2.rectangle(np_image, (x1, y1), (x2, y2), (255,0,0), 2)
    
    return np_image

In [ ]:
def custom_vision_predictor(file_path, prediction_key, endpoint, projectID,thresholding_values, iteration, predictions_per_sec=1, min_probability=50, debug=False):
    # Getting parameters from the path
    predictor = CustomVisionPredictionClient(prediction_key, endpoint=endpoint)
    file_info = get_file_info(file_path)

    # Split video into frames
    video_dir = os.path.join(file_info['year'], file_info['billabong_type'], file_info['site_name'], file_info['transect_name'],file_info['location_name']).replace(os.sep, '-').replace(' ', '-')
            
    starting_time = 0 # Seconds...

    video_capture = cv2.VideoCapture(file_path)

    num_of_frames = video_capture.get(cv2.CAP_PROP_FRAME_COUNT)

    fps = int(video_capture.get(cv2.CAP_PROP_FPS))

    if debug:
        print(f"Frames per second: {fps}")
        print(f"Total frame count: {video_capture.get(cv2.CAP_PROP_FRAME_COUNT)}")

    frame_count = int(starting_time * fps)

    video_capture.set(cv2.CAP_PROP_POS_FRAMES, frame_count)

    error_frame_count = 0
    
    writer = None

    # Analyse video frames and ran custom vision 
    while video_capture.isOpened():
        success, np_image = video_capture.read()

        if (frame_count % (fps//predictions_per_sec)) == 0:
            if success is False:
                print(f'Could not process frame: {frame_count} of {num_of_frames}')
                error_frame_count += 1
                frame_count += 1

                if frame_count == num_of_frames:
                    break
                else:
                    continue

            frame_name = '{0}_Frame-{1}.jpg'.format(video_dir, frame_count)

            np_image = cv2.cvtColor(np_image, cv2.COLOR_BGR2RGB)
            buffer = io.BytesIO()
            Image.fromarray(np_image).save(buffer, format='JPEG')
            results = predictor.detect_image(projectID, iteration, buffer.getvalue())
            
            if writer is None:
                fourcc = cv2.VideoWriter_fourcc(*'MP4V')
                writer = cv2.VideoWriter('{0}-Scored.MP4'.format(video_dir), fourcc, video_capture.get(cv2.CAP_PROP_FPS),(np_image.shape[1], np_image.shape[0]), True)
            
            np_image = draw_bounding_boxes(results,np_image,frame_count,fps,min_probability,debug)
            
            writer.write(np_image)

        frame_count += 1

        if frame_count == num_of_frames:
            break

    video_capture.release()
    
    writer.release()

    print(f"Total video frames:{num_of_frames}")
    print(f"Total frames to process: {num_of_frames/fps}")
    print(f"Total processed frames that errored: {error_frame_count}")
    # print(f'Total number of unprocessed frames: {error_frame_count} of {num_of_frames/fps}')

In [ ]:
files = os.listdir(path)

for f in files:
    if os.path.splitext(f)[1] == '.mp4':
        file_path = path + '\\' + f

        custom_vision_predictor(file_path, prediction_key, endpoint, projectID, thresholding_values, iteration, predictions_per_sec=1, min_probability=15, debug=False)